In [1]:
# PARSEADO DE LA PAGINA DE MINISTERIO DE SANIDAD Y CONSUMO, 
# antes solo conseguiamos los resultados de la primera pagina, analizando la pagina vemos que
# hay que añadir el parametro index

In [4]:
import requests
from bs4 import BeautifulSoup
import time

# Función adaptada del código anterior que buscaba solo en la primera página
def buscarDocsEnMSCBS(busqueda, sIndex=0):
    url = "https://www.mscbs.gob.es/buscador/iniciar.do"
    
    print("Buscando %s (%d)" %(busqueda, sIndex))

    parametros = {
        'search': busqueda,  
        'buscar': 'msc',    
        'sIndex': sIndex
    }

    respuesta = requests.post(url, parametros)

    html = respuesta.content.decode("utf-8")   
    soup = BeautifulSoup(html, 'html.parser')
    
    # Averiguamos el número total de resultados sacándolo de <div id="titulo_res">
    div = soup.find(id="titulo_res")
    
    if (div == None):
        return {"nResultados": 0, "articulos": []}  # No hemos encontrado nada
    
    nResultados = int(div.text.split(" ")[0])
    
    # Como en el ejemplo anterior obtenemos los títulos y URLs de los resultados
    
    cuadroResultados = soup.find('div', {"class": "capaCentroBuscador"})

    resultados = cuadroResultados.find_all('li')

    articulos = []
    
    for r in resultados:
        # Dentro de cada <li> tenemos un <p> con un <a href="URL_DE_LA_NOTICIA">
        urlNoticia = r.p.a['href']

        tituloNoticia = r.p.a.text

        articulos.append({'titulo': tituloNoticia, 'url': urlNoticia})
        
    return {"nResultados": nResultados, "articulos": articulos}

print("Introduzca lo que quiera buscar en el MSCBS:")
busqueda = input()
print("")

resultados = buscarDocsEnMSCBS(busqueda)

nResultados = resultados['nResultados']
articulos = resultados['articulos']

# Ahora tenemos que hacer búsquedas sucesivas para seguir obteniendo de 10 en 10
# el resto de artículos

for sIndex in range(10, nResultados, 10):
    time.sleep(0.5)      # Somo educados: no mandamos sucesivas peticiones sin dejar un poco
                    # de tiempo (1 segundo)
        
    resultados = buscarDocsEnMSCBS(busqueda, sIndex)
    
    articulos += resultados['articulos']
    

print("Se han encontrado %d artículos:" %(nResultados))
indice = 1

for a in articulos:
    print(str(indice) + " - " + a['titulo'])
    print("   " + a['url'])
    indice += 1
    

Introduzca lo que quiera buscar en el MSCBS:
covid19

Buscando covid19 (0)
Buscando covid19 (10)
Buscando covid19 (20)
Buscando covid19 (30)
Buscando covid19 (40)
Buscando covid19 (50)
Buscando covid19 (60)
Buscando covid19 (70)
Buscando covid19 (80)
Buscando covid19 (90)
Buscando covid19 (100)
Buscando covid19 (110)
Buscando covid19 (120)
Buscando covid19 (130)
Se han encontrado 136 artículos:
1 - [Ministerio de Sanidad, Consumo y Bienestar Social - Profesionales - Información para la ciudadanía - Coronavirus]
   https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/ciudadania.htm
2 - [Ministerio de Sanidad, Consumo y Bienestar Social - Profesionales -
Ruedas de prensa - COVID19]
   https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/videosPrensa.htm
3 - [Ministerio de Sanidad, Consumo y Bienestar Social - Profesionales -
Ruedas de prensa - COVID19]
   https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual

In [ ]:
# ROBOTS RECURSIVOS

In [8]:
from urllib.parse import urlparse    # Utilizadas para parsear más fácilmente las URLs que encontremos.
from urllib.parse import urlunparse  #
from urllib.parse import urljoin     #

import requests
from bs4 import BeautifulSoup
import time
import shutil                        # Una utilidad que permite copiar ficheros facilmente
import os
import errno


retardoEntrePeticiones = 0.5

# A esta función se le pasa una URL padre "completa" (con esquema y netloc: 
# http://example.com...)
# y una "hija" que probablemente se habrá sacado de un <a href="">. La hija probablemente
# no tenga ni esquema ni netloc. Esta función se asegura de devolver la hija pero
# con el esquema y el netloc correspondientes. Si la hija ya tiene netloc la devolverá tal cual.
# Ejemplos:
#  Padre: "https://etsiit.ugr.es/pages/calendario_academico" 
#  Hija: "horariosnuevocurso"
#  Devuelve: "https://etsiit.ugr.es/pages/calendario_academico/horariosnuevocurso"
#
#  Padre: "https://etsiit.ugr.es/pages/escuela"
#  Hija: "/pages/docencia"
#  Devuelve: https://etsiit.ugr.es/pages/docencia
#  
#  Padre: "https://etsiit.ugr.es/pages/escuela"
#  Hija: "https://www.ugr.es"
#  Devuelve: "https://www.ugr.es"
# 
# Ojo, no está probada con parámetros y similares 

def getURLCompleta(padreURL, hijaURL):
    hija = urlparse(hijaURL)
    padre = urlparse(padreURL)
    
    netloc = hija.netloc
    scheme = hija.scheme
    path = hija.path
    
    if (netloc == ''):   # Si no tiene netloc, se lo añadimos
        netloc = padre.netloc
        scheme = padre.scheme
        
        path = urljoin(padre.path, hija.path)
    
    return urlunparse((scheme, netloc, path, hija.params, hija.query, hija.fragment))
    
# Devuelve True si las dos urls son del mismo servidor
def sonDelMismoServidor(url1, url2):
    url1p = urlparse(url1)
    url2p = urlparse(url2)
    
    server1 = url1p.netloc.split(".")[-2:]
    server2 = url2p.netloc.split(".")[-2:]
    
    if server1[0]==server2[0] and server1[1]==server2[1]:
        return True
    
    return False

# Crea todos los directorios necesarios en el path de un fichero
def crearDirectoriosDeUnPath(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    
    

def descargaImagen(url, directorio):
    urlp = urlparse(url)
    
    nombreFichero = directorio + urlp.netloc + urlp.path
    #print(nombreFichero)
    
    if not os.path.exists(nombreFichero):  # Si ya existe es que lo descargamos en una ejecución anterior
        time.sleep(retardoEntrePeticiones)   # Dejamos un tiempo por educación

        crearDirectoriosDeUnPath(nombreFichero)

        print("Descargando imagen %s" % (url))

        resp = requests.get(url, stream=True)

        local_file = open(nombreFichero, 'wb')
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
        shutil.copyfileobj(resp.raw, local_file)
    # Remove the image url response object.
        del resp
        
        return True
    
    return False

# Preguntamos la web a escrapear y otros parámetros
print("Inserte la URL a scrapear:")
url = input("Por ejemplo: https://wallpaperscraft.com/ ")
if url == '': url = "https://wallpaperscraft.com/"
    
print("\nMáximo nivel de profundidad en la búsqueda:")
profundidadMaxima = input("Por ejemplo: 1 ")
if profundidadMaxima == '': 
    profundidadMaxima = 1 
else: 
    profundidadMaxima = int(profundidadMaxima)

print("\nDirectorio donde guardar las imágenes:")
directorioImagenes = input("Por ejemplo: /home/zerjillo/temp/imagenes/ ")
if directorioImagenes == '':
    directorioImagenes =  "/home/diego/"


# Creamos arrays para las URLs que todavía tenemos que parsear y las que ya hemos visitado
# Las visitadas las apuntamos para evitar descargar dos veces la misma URL.
urlsPendientes = [{'url' : url, 'profundidad' : 1}]
urlsVisitadas = []  
imagenesDescargadas = 0

# Mientras que queden URLs por parsear
while(len(urlsPendientes) > 0):
    x = urlsPendientes.pop()
    
    urlActual = x['url']
    profundidadActual = x['profundidad']
    
    urlsVisitadas.append(urlActual)  # Para no visitar de nuevo esta misma URL
    
    time.sleep(retardoEntrePeticiones)    # Dejamos un tiempo por educación
    
    print("Descargando página %s" % (urlActual))
    
    respuesta = requests.get(urlActual)

    try:
        html = respuesta.content.decode("utf-8")   
        soup = BeautifulSoup(html, 'html.parser')


    # Si no hemos llegado al nivel de profundidad maximo, buscamos todos los enlaces y los mete
        if profundidadActual < profundidadMaxima:
            enlaces = soup.findAll('a')

            for enlace in enlaces:
                urlEnlace = enlace['href']

                if urlEnlace != '':
                    urlEnlace = getURLCompleta(urlActual, urlEnlace)

                    if sonDelMismoServidor(urlActual, urlEnlace) and not urlEnlace in urlsVisitadas and not urlEnlace in urlsPendientes:
                        #print(urlEnlace)
                        urlsPendientes.append({'url': urlEnlace, 'profundidad': profundidadActual + 1})


    # Descargamos las imágenes
        imagenes = soup.findAll('img')


        for imagen in imagenes:
            urlImagen = imagen['src']

            if urlImagen != '':
                urlImagen = getURLCompleta(urlActual, urlImagen)

                if not urlImagen in urlsVisitadas:
                    if descargaImagen(urlImagen, directorioImagenes):
                        print("Quedan %d páginas, imagenes descargadas %d" % (len(urlsPendientes), imagenesDescargadas))

                        imagenesDescargadas += 1
                        
                    urlsVisitadas.append(urlImagen)
                    
           
        del respuesta
    except Exception as e: 
        print(e)
        print("Problema parsean página ¿binario?")

Inserte la URL a scrapear:
Por ejemplo: https://wallpaperscraft.com/ 

Máximo nivel de profundidad en la búsqueda:
Por ejemplo: 1 

Directorio donde guardar las imágenes:
Por ejemplo: /home/zerjillo/temp/imagenes/ 
Descargando página https://wallpaperscraft.com/
name 'urlparse' is not defined
Problema parsean página ¿binario?
